<a href="https://colab.research.google.com/github/eTcilopp/geek_brains/blob/main/044_machine_learning/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10

1. Загрузите набор данных, подходящий для задачи классификации или регрессии. Вы можете использовать любой набор данных, предоставленный в библиотеке scikit-learn или другом доступном источнике данных.

2. Разделите данные на обучающую и тестовую выборки с помощью функции train_test_split из библиотеки scikit-learn.

3. Создайте модель машинного обучения, которую вы хотите использовать для предсказания на этом наборе данных. Например, можно использовать модель линейной регрессии или случайный лес для задачи регрессии, или модель логистической регрессии или случайный лес для задачи классификации.

4. Используйте метод кросс-валидации, чтобы оценить производительность модели на обучающей выборке. Например, можно использовать метод cross_val_score из библиотеки scikit-learn с использованием метрики оценки, такой как точность (accuracy) или среднеквадратичная ошибка (mean squared error).

5. Проведите подбор гиперпараметров модели с помощью метода GridSearchCV из библиотеки scikit-learn. Этот метод позволяет перебирать различные значения гиперпараметров и выбирать наилучшие значения на основе кросс-валидации.

6. Обучите модель с использованием оптимальных гиперпараметров на обучающей выборке и оцени ее производительность на тестовой выборке.

7. Проведите анализ результатов и сделай выводы о процессе кросс-валидации и подборе гиперпараметров в задачах машинного обучения.

In [1]:
!pip install yfinance -q

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
import yfinance as yf
from datetime import datetime, timedelta

В качестве датасета возьмем историю цен на акции компании AAPL и попробуем предсказать восходящий тренд в течение следующих трех дней.

In [4]:
TICKER = 'AAPL'
YEARS_HISTORY = 2
start_date = datetime.now() - timedelta(days=(365 * YEARS_HISTORY))
df = yf.download(TICKER, start=start_date, end=datetime.now())

[*********************100%***********************]  1 of 1 completed


In [5]:
df.head(2)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2022-12-20,130.923233,132.300003,133.250000,129.889999,131.389999,77432800
2022-12-21,134.040451,135.449997,136.809998,132.750000,132.979996,85928000


In [6]:
df['High Plus 1 Day'] = df['High'].shift(-1)
df['High Plus 2 Day'] = df['High'].shift(-2)
df['High Plus 3 Day'] = df['High'].shift(-3)
df['Low Plus 1 Day'] = df['Low'].shift(-1)
df['Low Plus 2 Day'] = df['Low'].shift(-2)
df['Low Plus 3 Day'] = df['Low'].shift(-3)

Добавим Target

In [7]:
df['Target'] = np.where(
    (
        (df[('High Plus 1 Day', '')] > df[('High', 'AAPL')]) &
        (df[('High Plus 2 Day', '')] > df[('High Plus 1 Day', '')]) &
        (df[('High Plus 3 Day', '')] > df[('High Plus 2 Day', '')]) &
        (df[('Low Plus 1 Day', '')] > df[('Low', 'AAPL')]) &
        (df[('Low Plus 2 Day', '')] > df[('Low Plus 1 Day', '')]) &
        (df[('Low Plus 3 Day', '')] > df[('Low Plus 2 Day', '')])
    ),
    1,
    0
)


In [8]:
df['Target'].value_counts()

,count
Target,
0,432
1,71


Классы получились сильно не сбалансированными. При создании модели нужно будет это учесть. <br>
Уберем "промежуточные" значения из будущего, которого у нас не будет

In [9]:
df = df.drop(columns=['High Plus 1 Day', 'High Plus 2 Day', 'High Plus 3 Day', 'Low Plus 1 Day', 'Low Plus 2 Day', 'Low Plus 3 Day'])

In [10]:
df.head(2)

Price,Adj Close,Close,High,Low,Open,Volume,Target
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,,
2022-12-20,130.923233,132.300003,133.250000,129.889999,131.389999,77432800,0
2022-12-21,134.040451,135.449997,136.809998,132.750000,132.979996,85928000,0


Добавим скользящие средние значения цен за 10, 50 и 200 дней.

In [11]:
df['SMA 10'] = df['Close'].rolling(window=10).mean()
df['SMA 50'] = df['Close'].rolling(window=50).mean()
df['SMA 200'] = df['Close'].rolling(window=200).mean()

Преимуществом такого датасета является его "чистота": можно быть уверенным, что все данные корректны и присутствуют.

In [12]:
df = df.dropna()

Преобразуем даты в дополнительный набор параметров: <br>
- номер дня недели;<br>
- номер месяца.<br>
Год отбросим за ненадобностью

In [13]:
df = df.reset_index()
df['Day of the week'] = df['Date'].dt.day_of_week
df['Month'] = df['Date'].dt.month

df = df.drop(columns=['Date'], level=0)

df.head()

Price,Adj Close,Close,High,Low,Open,Volume,Target,SMA 10,SMA 50,SMA 200,Day of the week,Month
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,
0,176.392319,177.490005,177.990005,173.179993,173.800003,57224100,0,173.258002,179.389401,167.12280,4,10
1,177.883057,178.990005,179.050003,175.800003,176.809998,42390800,0,173.549002,179.052601,167.35625,0,10
2,177.286758,178.389999,179.720001,177.949997,178.100006,43698000,0,174.192001,178.691401,167.57095,1,10
3,178.688034,179.800003,179.850006,177.600006,178.199997,47551100,0,175.129002,178.375201,167.80880,2,10
4,179.592422,180.710007,182.339996,179.039993,180.070007,56743100,0,176.131003,178.137801,168.05305,3,10


Отмасштабируем данные

In [14]:
s_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA 10', 'SMA 50', 'SMA 200']
scaler = StandardScaler()
df[s_columns] = scaler.fit_transform(df[s_columns])

Получим модель и проверим ее на тестовых данных

In [15]:
X = df.drop(columns=['Target'])
y = df['Target']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
rf = RandomForestClassifier(random_state=42)
model_base = rf.fit(X_train, y_train)

In [18]:
y_train_pred = model_base.predict(X_train)

precision = precision_score(y_train, y_train_pred)
recall = recall_score(y_train, y_train_pred)
f1 = f1_score(y_train, y_train_pred)
accuracy = accuracy_score(y_train, y_train_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0


In [19]:
y_test_pred = model_base.predict(X_test)

precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
accuracy = accuracy_score(y_test, y_test_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 1.0
Recall: 0.3333333333333333
F1 Score: 0.5
Accuracy: 0.9016393442622951


Идеальные результаты на обучающей выборке и откровенно плохие на тестовой.<br>
Модель явно переобучена. Попробуем подобрать оптимальные гиперпараметры.

In [20]:
param_grid = {
    'n_estimators': [50, 100, 150],           # Количество деревьев
    'max_depth': [None, 10, 20],              # Максимальная глубина дерева
    'min_samples_split': [2, 5, 10],          # Минимальное количество образцов для разделения
    'min_samples_leaf': [1, 2, 4],            # Минимальное количество листьев
    'max_features': ['sqrt', 'log2', None]    # Количество признаков для использования
}

In [21]:
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='f1',        # Метрика для оценки
    cv=3,                      # Количество фолдов кросс-валидации
    verbose=2,                 # Уровень логов
    n_jobs=-1                  # Параллелизм
)

In [22]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='f1', verbose=2)

In [23]:
rf_model = grid_search.best_estimator_

In [24]:
y_test_pred = rf_model.predict(X_test)

In [25]:
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
accuracy = accuracy_score(y_test, y_test_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 1.0
Recall: 0.2222222222222222
F1 Score: 0.36363636363636365
Accuracy: 0.8852459016393442


Получили стопроцентную точность: во всех случаях предсказаний 3-хдневного роста этот рост случился. <br>
При этом Recall получился низким: модель пропустила 78% случаев. <br>
Это хорошо с точки зрения риска: шанс потерять вложения минимален, однако, мы имеем упущенную выгоду: пропускаем значительную долю ситуаций роста. <br>
Попробуем оптимизировать набор признаков: применим "Исчерпывающий отбор признаков".

In [26]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector

efs = ExhaustiveFeatureSelector(RandomForestClassifier(),
        min_features=4,
        max_features=11,
        scoring='roc_auc',
        cv=2)
efs = efs.fit(X, y)

Features: 1816/1816

In [27]:
selected_features = X_train.columns[list(efs.best_idx_)]
print(selected_features)

MultiIndex([(          'Close', 'AAPL'),
            (         'Volume', 'AAPL'),
            (         'SMA 50',     ''),
            ('Day of the week',     '')],
           names=['Price', 'Ticker'])


Построим модель только по этим признакам

In [28]:
X = df[selected_features]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
rf = RandomForestClassifier(random_state=42)
model_base = rf.fit(X_train, y_train)

In [30]:
y_train_pred = model_base.predict(X_train)

precision = precision_score(y_train, y_train_pred)
recall = recall_score(y_train, y_train_pred)
f1 = f1_score(y_train, y_train_pred)
accuracy = accuracy_score(y_train, y_train_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0


In [31]:
y_test_pred = model_base.predict(X_test)

precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
accuracy = accuracy_score(y_test, y_test_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.5
Recall: 0.1111111111111111
F1 Score: 0.18181818181818182
Accuracy: 0.8524590163934426


Подберем гиперпараметры (воспользуемся настройками, используемыми ранее)

In [32]:
grid_search.fit(X_train, y_train)
rf_model = grid_search.best_estimator_

Fitting 3 folds for each of 243 candidates, totalling 729 fits


In [33]:
y_test_pred = rf_model.predict(X_test)

precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
accuracy = accuracy_score(y_test, y_test_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.3333333333333333
Recall: 0.1111111111111111
F1 Score: 0.16666666666666666
Accuracy: 0.8360655737704918


Результаты еще ухудшились. Очевидно, оптимальными явились настройки "по умолчанию" - <br>
с Precision = 100% и Recall = 22% вполне можно спекулировать с прибылью, теряя при этом 78% возможностей, что приемлемо в парадигме консервативного трейдинга.

# Выводы:
методы кросс-валидации и подбора параметров - мощный инструмент оптимизации моделей, тем не менее, не всегда работает в реальном нелинейном мире. Тем не менее, разбираться в методах оптимизации моделей необходимо.